In [8]:
# Install required packages
!pip install numpy pandas matplotlib scikit-learn seaborn

# Building Intuition for Survey Estimators: A Simple Simulation

## The Fundamental Insight: "Predict-and-Correct"

**Every survey estimator follows the same basic formula:**

$$\text{Total Estimate} = \text{Predicted Total} + \text{Correction for Prediction Error}$$

This notebook demonstrates this core concept using a simple, intuitive example.

## Scenario: Estimating Total Business Revenue in a Region

We'll use a business revenue example with an obvious, intuitive relationship:

- **Population**: 1,000 businesses in a region
- **Sample**: We survey only 50 businesses (5%)
- **Target variable (Y)**: Annual revenue ($)
- **Auxiliary variable (X)**: Number of employees (known for all businesses from business registry)
- **Intuitive relationship**: More employees → Higher revenue

### Why This Example Works Well

1. The relationship is obvious: larger businesses (more employees) generate more revenue
2. It's realistic: business registries often have employee counts but not revenue
3. The numbers are interpretable: ~$50,000 revenue per employee is reasonable

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor

# Set random seed for reproducibility
np.random.seed(42)

# Make plots look better
plt.style.use('seaborn-v0_8-darkgrid')
plt.rcParams['figure.dpi'] = 100

## Step 1: Create the Population of Businesses

In [ ]:
# Population parameters
N = 1000  # Total number of businesses in the region
n = 50    # Number of businesses we'll sample

# Generate business data
# Number of employees: small businesses (1-5), medium (6-20), large (21-50)
employees = np.concatenate([
    np.random.randint(1, 6, 600),      # 60% small businesses
    np.random.randint(6, 21, 300),     # 30% medium businesses  
    np.random.randint(21, 51, 100)     # 10% large businesses
])

# Revenue relationship: roughly $50,000 per employee + noise
# This is intuitive: more employees = more capacity = more revenue
base_revenue_per_employee = 50000
revenue = base_revenue_per_employee * employees + np.random.normal(0, 20000, N) * employees

# Create population dataframe
population = pd.DataFrame({
    'business_id': range(N),
    'employees': employees,
    'revenue': revenue
})

# TRUE TOTAL we want to estimate
TRUE_TOTAL = population['revenue'].sum()
print(f"True total revenue for all businesses: ${TRUE_TOTAL:,.0f}")
print(f"Average revenue per business: ${TRUE_TOTAL/N:,.0f}")
print(f"\nPopulation summary:")
print(population.describe())

## Step 2: Take a Random Sample

In practice, we can only afford to survey a small fraction of businesses.

In [ ]:
# Take a simple random sample
sample_idx = np.random.choice(N, n, replace=False)
sample = population.iloc[sample_idx].copy()
sample['weight'] = N/n  # Each sampled business represents N/n businesses

print(f"Sample size: {n} businesses ({n/N*100:.0f}% of population)")
print(f"\nSample summary:")
print(sample[['employees', 'revenue']].describe())

## Step 3: Apply Each Estimator

Now we'll see how each estimator uses the same "predict-and-correct" logic with different approaches to prediction.

### 3.1 Difference Estimator

**Key assumption**: We KNOW the exact revenue-per-employee ratio (e.g., from last year's census or industry studies).

- **Prediction model**: Revenue = $50,000 × Employees (known beforehand)
- **When it works well**: When the known ratio is accurate and stable over time
- **When it fails**: When the true relationship has changed

In [ ]:
# Difference Estimator
KNOWN_RATIO = 50000  # Assume we know this from previous studies

# Predict revenue for entire population using known ratio
predicted_revenue_diff = population['employees'] * KNOWN_RATIO
predicted_total_diff = predicted_revenue_diff.sum()

# Calculate prediction errors in sample
sample['prediction_diff'] = sample['employees'] * KNOWN_RATIO
sample['error_diff'] = sample['revenue'] - sample['prediction_diff']

# Correction = weighted sum of errors
correction_diff = (sample['error_diff'] * sample['weight']).sum()

# Final estimate
estimate_diff = predicted_total_diff + correction_diff

print(f"=== DIFFERENCE ESTIMATOR ===")
print(f"Known ratio: ${KNOWN_RATIO:,} per employee")
print(f"Predicted total: ${predicted_total_diff:,.0f}")
print(f"Correction from sample: ${correction_diff:,.0f}")
print(f"Final estimate: ${estimate_diff:,.0f}")
print(f"Error vs true total: ${estimate_diff - TRUE_TOTAL:,.0f} ({(estimate_diff - TRUE_TOTAL)/TRUE_TOTAL*100:.1f}%)")

### 3.2 GREG Estimator

**Key feature**: LEARNS a linear relationship from the sample data.

- **Prediction model**: Revenue = β₀ + β₁ × Employees (coefficients estimated from sample)
- **Main advantage**: Robust to model misspecification, achieves calibration
- **NSO favorite**: Used in production systems like Statistics Canada's GES

In [13]:
# GREG Estimator
greg_model = LinearRegression()
greg_model.fit(sample[['employees']], sample['revenue'], sample_weight=sample['weight'])

# Predict for entire population
predicted_revenue_greg = greg_model.predict(population[['employees']])
predicted_total_greg = predicted_revenue_greg.sum()

# Calculate errors in sample
sample['prediction_greg'] = greg_model.predict(sample[['employees']])
sample['error_greg'] = sample['revenue'] - sample['prediction_greg']

# Correction
correction_greg = (sample['error_greg'] * sample['weight']).sum()

# Final estimate
estimate_greg = predicted_total_greg + correction_greg

print(f"=== GREG ESTIMATOR ===")
print(f"Learned relationship: Revenue = ${greg_model.intercept_:,.0f} + ${greg_model.coef_[0]:,.0f} × Employees")
print(f"Predicted total: ${predicted_total_greg:,.0f}")
print(f"Correction from sample: ${correction_greg:,.0f}")
print(f"Final estimate: ${estimate_greg:,.0f}")
print(f"Error vs true total: ${estimate_greg - TRUE_TOTAL:,.0f} ({(estimate_greg - TRUE_TOTAL)/TRUE_TOTAL*100:.1f}%)")

=== GREG ESTIMATOR ===
Learned relationship: Revenue = $38,407 + $39,592 × Employees
Predicted total: $403,956,547
Correction from sample: $-0
Final estimate: $403,956,547
Error vs true total: $-75,708,514 (-15.8%)


### 3.3 PPI Estimator (using Machine Learning)

**Key innovation**: Can use ANY prediction model, including complex ML algorithms.

- **Prediction model**: Random Forest (or any ML model)
- **Main advantage**: Can capture complex, non-linear relationships
- **Trade-off**: No calibration guarantees, requires careful model validation

In [ ]:
# PPI Estimator using Random Forest
# In practice, this would be trained on a larger labeled dataset
ppi_model = RandomForestRegressor(n_estimators=10, random_state=42)
ppi_model.fit(sample[['employees']], sample['revenue'])

# Predict for entire population
predicted_revenue_ppi = ppi_model.predict(population[['employees']])
predicted_total_ppi = predicted_revenue_ppi.sum()

# Calculate errors in sample
sample['prediction_ppi'] = ppi_model.predict(sample[['employees']])
sample['error_ppi'] = sample['revenue'] - sample['prediction_ppi']

# Correction
correction_ppi = (sample['error_ppi'] * sample['weight']).sum()

# Final estimate
estimate_ppi = predicted_total_ppi + correction_ppi

print(f"=== PPI ESTIMATOR ===")
print(f"Model: Random Forest with {ppi_model.n_estimators} trees")
print(f"Predicted total: ${predicted_total_ppi:,.0f}")
print(f"Correction from sample: ${correction_ppi:,.0f}")
print(f"Final estimate: ${estimate_ppi:,.0f}")
print(f"Error vs true total: ${estimate_ppi - TRUE_TOTAL:,.0f} ({(estimate_ppi - TRUE_TOTAL)/TRUE_TOTAL*100:.1f}%)")

## Step 4: Visualize the Core Concept

This visualization shows how ALL estimators follow the same "predict-and-correct" pattern.

In [ ]:
# Create visualization showing predict-and-correct decomposition
fig, ax = plt.subplots(figsize=(12, 8))

estimators = ['True Total', 'Difference', 'GREG', 'PPI']
predictions = [0, predicted_total_diff, predicted_total_greg, predicted_total_ppi]
corrections = [TRUE_TOTAL, correction_diff, correction_greg, correction_ppi]
totals = [TRUE_TOTAL, estimate_diff, estimate_greg, estimate_ppi]

x = np.arange(len(estimators))
width = 0.6

# Create stacked bar chart
bars1 = ax.bar(x, predictions, width, label='Predicted Total', color='lightblue', edgecolor='black')
bars2 = ax.bar(x, corrections, width, bottom=predictions, label='Correction', color='orange', edgecolor='black')

# Add total estimate values on top
for i, (pred, corr, total) in enumerate(zip(predictions, corrections, totals)):
    ax.text(i, total + 50000, f'${total/1e6:.1f}M', ha='center', va='bottom', fontweight='bold', fontsize=12)
    
    # Add prediction and correction values inside bars
    if i > 0:  # Skip true total
        ax.text(i, pred/2, f'${pred/1e6:.1f}M', ha='center', va='center', fontsize=10)
        ax.text(i, pred + corr/2, f'${corr/1e6:+.1f}M', ha='center', va='center', fontsize=10)

ax.set_ylabel('Revenue ($)', fontsize=12)
ax.set_title('The Core Logic of All Survey Estimators\n' + 
             'Total Estimate = Predicted Total + Correction for Prediction Error',
             fontsize=14, fontweight='bold')
ax.set_xticks(x)
ax.set_xticklabels(estimators, fontsize=12)
ax.legend(fontsize=12, loc='upper right')

# Add horizontal line at true total
ax.axhline(y=TRUE_TOTAL, color='red', linestyle='--', alpha=0.7, linewidth=2)

# Format y-axis to show millions
ax.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'${x/1e6:.0f}M'))

plt.tight_layout()
plt.show()

## Key Insights Summary

### The Universal Formula
All estimators follow: **Total Estimate = Predicted Total + Correction**

### What Makes Each Estimator Different?

In [ ]:
# Create summary table
summary_data = {
    'Estimator': ['Difference', 'GREG', 'PPI'],
    'Prediction Model': ['Known: $50,000/employee', 'Learned: Linear regression', 'Learned: Random Forest'],
    'Predicted Total': [f'${predicted_total_diff/1e6:.1f}M', 
                       f'${predicted_total_greg/1e6:.1f}M', 
                       f'${predicted_total_ppi/1e6:.1f}M'],
    'Correction': [f'${correction_diff/1e6:+.1f}M', 
                  f'${correction_greg/1e6:+.1f}M', 
                  f'${correction_ppi/1e6:+.1f}M'],
    'Final Estimate': [f'${estimate_diff/1e6:.1f}M', 
                      f'${estimate_greg/1e6:.1f}M', 
                      f'${estimate_ppi/1e6:.1f}M'],
    'Error': [f'{(estimate_diff - TRUE_TOTAL)/TRUE_TOTAL*100:.1f}%', 
             f'{(estimate_greg - TRUE_TOTAL)/TRUE_TOTAL*100:.1f}%', 
             f'{(estimate_ppi - TRUE_TOTAL)/TRUE_TOTAL*100:.1f}%']
}

summary_df = pd.DataFrame(summary_data)
print(f"True Total: ${TRUE_TOTAL/1e6:.1f}M\n")
summary_df

## Before Moving On: Key Questions to Test Your Understanding

1. **Why do we need the correction term?**
   - Because our prediction model is never perfect. The correction ensures unbiasedness.

2. **What happens if we skip the correction?**
   - We get a biased estimate. Even with a great model, small systematic errors accumulate over the population.

3. **When would the Difference Estimator be best?**
   - When you have reliable external information about the true relationship (e.g., last year's census).

4. **Why do NSOs love GREG?**
   - It's robust (works even if model is wrong) and achieves calibration (sample totals match known population totals).

5. **What's the main advantage of PPI?**
   - Can use powerful ML models for better predictions, potentially achieving much lower variance.

6. **What's the main limitation of PPI?**
   - No calibration guarantees. Different ML model needed for each variable. Less transparent for official statistics.

## Next Steps

Now that you understand the core "predict-and-correct" logic, you can explore:

1. **Model-Calibrated Estimators**: How they generalize GREG to non-linear models while keeping calibration
2. **Variance estimation**: How each estimator's variance depends on model quality
3. **Multi-purpose surveys**: Why calibration matters when estimating many variables
4. **CrossPPI**: How to avoid overfitting when using the same data for training and correction